In [6]:
import os
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from sb3_contrib import MaskablePPO
from stable_baselines3.common.policies import ActorCriticPolicy
import gymnasium as gym
from gymnasium import spaces
from datetime import datetime
import glob
import torchOptics.optics as tt
import torch.nn as nn
import torchOptics.metrics as tm
import torch.nn.functional as F
import torch.optim
import torch
from torch.utils.data import Dataset, DataLoader
import os
import glob
import matplotlib.pyplot as plt
import pickle
import torchvision
import tqdm
import time
import pandas as pd
from sb3_contrib.common.maskable.utils import get_action_masks
from sb3_contrib.common.wrappers import ActionMasker
from stable_baselines3 import PPO

# 현재 날짜와 시간을 가져와 포맷 지정
current_date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

torch.backends.cudnn.enabled = False

class BinaryNet(nn.Module):
    def __init__(self, num_hologram, final='Sigmoid', in_planes=3,
                 channels=[32, 64, 128, 256, 512, 1024, 2048, 4096],
                 convReLU=True, convBN=True, poolReLU=True, poolBN=True,
                 deconvReLU=True, deconvBN=True):
        super(BinaryNet, self).__init__()

        def CRB2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True, relu=True, bn=True):
            layers = []
            layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                 kernel_size=kernel_size, stride=stride, padding=padding,
                                 bias=bias)]
            if relu:
                layers += [nn.Tanh()]
            if bn:
                layers += [nn.BatchNorm2d(num_features=out_channels)]

            cbr = nn.Sequential(*layers)  # *으로 list unpacking

            return cbr

        def TRB2d(in_channels, out_channels, kernel_size=2, stride=2, bias=True, relu=True, bn=True):
            layers = []
            layers += [nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels,
                                          kernel_size=2, stride=2, padding=0,
                                          bias=True)]
            if bn:
                layers += [nn.BatchNorm2d(num_features=out_channels)]
            if relu:
                layers += [nn.ReLU()]

            cbr = nn.Sequential(*layers)  # *으로 list unpacking

            return cbr

        self.enc1_1 = CRB2d(in_planes, channels[0], relu=convReLU, bn=convBN)
        self.enc1_2 = CRB2d(channels[0], channels[0], relu=convReLU, bn=convBN)
        self.pool1 = CRB2d(channels[0], channels[0], stride=2, relu=poolReLU, bn=poolBN)

        self.enc2_1 = CRB2d(channels[0], channels[1], relu=convReLU, bn=convBN)
        self.enc2_2 = CRB2d(channels[1], channels[1], relu=convReLU, bn=convBN)
        self.pool2 = CRB2d(channels[1], channels[1], stride=2, relu=poolReLU, bn=poolBN)

        self.enc3_1 = CRB2d(channels[1], channels[2], relu=convReLU, bn=convBN)
        self.enc3_2 = CRB2d(channels[2], channels[2], relu=convReLU, bn=convBN)
        self.pool3 = CRB2d(channels[2], channels[2], stride=2, relu=poolReLU, bn=poolBN)

        self.enc4_1 = CRB2d(channels[2], channels[3], relu=convReLU, bn=convBN)
        self.enc4_2 = CRB2d(channels[3], channels[3], relu=convReLU, bn=convBN)
        self.pool4 = CRB2d(channels[3], channels[3], stride=2, relu=poolReLU, bn=poolBN)

        self.enc5_1 = CRB2d(channels[3], channels[4], relu=convReLU, bn=convBN)
        self.enc5_2 = CRB2d(channels[4], channels[4], relu=convReLU, bn=convBN)

        self.deconv4 = TRB2d(channels[4], channels[3], relu=deconvReLU, bn=deconvBN, stride=2)
        self.dec4_1 = CRB2d(channels[4], channels[3], relu=convReLU, bn=convBN)
        self.dec4_2 = CRB2d(channels[3], channels[3], relu=convReLU, bn=convBN)

        self.deconv3 = TRB2d(channels[3], channels[2], relu=deconvReLU, bn=deconvBN, stride=2)
        self.dec3_1 = CRB2d(channels[3], channels[2], relu=convReLU, bn=convBN)
        self.dec3_2 = CRB2d(channels[2], channels[2], relu=convReLU, bn=convBN)

        self.deconv2 = TRB2d(channels[2], channels[1], relu=deconvReLU, bn=deconvBN, stride=2)
        self.dec2_1 = CRB2d(channels[2], channels[1], relu=convReLU, bn=convBN)
        self.dec2_2 = CRB2d(channels[1], channels[1], relu=convReLU, bn=convBN)

        self.deconv1 = TRB2d(channels[1], channels[0], relu=deconvReLU, bn=deconvBN, stride=2)
        self.dec1_1 = CRB2d(channels[1], channels[0], relu=convReLU, bn=convBN)
        self.dec1_2 = CRB2d(channels[0], channels[0], relu=convReLU, bn=convBN)

        self.classifier = CRB2d(channels[0], num_hologram, relu=False, bn=False)

    def forward(self, x):
        # Encoder
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        pool1 = self.pool1(enc1_2)

        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2 = self.pool2(enc2_2)

        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3 = self.pool3(enc3_2)

        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4 = self.pool4(enc4_2)

        enc5_1 = self.enc5_1(pool4)
        enc5_2 = self.enc5_2(enc5_1)

        deconv4 = self.deconv4(enc5_2)
        concat4 = torch.cat((deconv4, enc4_2), dim=1)
        dec4_1 = self.dec4_1(concat4)
        dec4_2 = self.dec4_2(dec4_1)

        deconv3 = self.deconv3(dec4_2)
        concat3 = torch.cat((deconv3, enc3_2), dim=1)
        dec3_1 = self.dec3_1(concat3)
        dec3_2 = self.dec3_2(dec3_1)

        deconv2 = self.deconv2(dec3_2)
        concat2 = torch.cat((deconv2, enc2_2), dim=1)
        dec2_1 = self.dec2_1(concat2)
        dec2_2 = self.dec2_2(dec2_1)

        deconv1 = self.deconv1(dec2_2)
        concat1 = torch.cat((deconv1, enc1_2), dim=1)
        dec1_1 = self.dec1_1(concat1)
        dec1_2 = self.dec1_2(dec1_1)

        # Final classifier
        out = self.classifier(dec1_2)
        out = nn.Sigmoid()(out)
        return out


model = BinaryNet(num_hologram=8, in_planes=1, convReLU=False,
                  convBN=False, poolReLU=False, poolBN=False,
                  deconvReLU=False, deconvBN=False).cuda()
test = torch.randn(1, 1, 1024, 1024).cuda()
out = model(test)
print(out.shape)


# 간단한 데이터셋 클래스
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, target_dir, meta, transform=None, isTrain=True, padding=0):
        """
        target_dir: 데이터가 저장된 디렉토리 경로
        meta: 이미지 메타 정보
        transform: 이미지 전처리
        isTrain: 학습용(True) 또는 검증용(False) 데이터셋 여부
        padding: 이미지 패딩
        """
        self.target_list = sorted(glob.glob(target_dir + '*.png'))
        self.meta = meta
        self.transform = transform
        self.isTrain = isTrain
        self.padding = padding
        self.random_crop = transforms.RandomCrop((1024, 1024))
        self.center_crop = transforms.CenterCrop(1024)

    def __len__(self):
        return len(self.target_list)

    def __getitem__(self, idx):
        # 이미지 로드
        img = tt.imread(self.target_list[idx], meta=self.meta, gray=True).unsqueeze(0)

        # 크기 조정
        if img.shape[-2] < 1024 or img.shape[-1] < 1024:
            img = transforms.Resize((1024, 1024))(img)

        # 학습용 랜덤 크롭 또는 검증용 센터 크롭
        if self.isTrain:
            img = self.random_crop(img)
        else:
            img = self.center_crop(img)

        # 패딩 추가
        img = transforms.functional.pad(
            img, (self.padding, self.padding, self.padding, self.padding)
        )

        # 변환 적용 (optional)
        if self.transform:
            img = self.transform(img)

        return img


# BinaryHologramEnv 클래스
class BinaryHologramEnv(gym.Env):
    def __init__(self, target_function, trainloader, meta, max_steps=1000000, T_PSNR=30, T_steps=100):
        """
        target_function: 타겟 이미지와의 손실(MSE 또는 PSNR) 계산 함수.
        trainloader: 학습 데이터셋 로더.
        max_steps: 최대 타임스텝 제한.
        T_PSNR: 목표 PSNR 값.
        T_steps: PSNR 목표를 유지해야 하는 최소 타임스텝.
        """
        super(BinaryHologramEnv, self).__init__()

        # 관찰 공간 (1, 8, 1024, 1024)
        self.observation_space = spaces.Box(low=0, high=1, shape=(1, 8, 1024, 1024), dtype=np.float32)

        # 행동 공간: MultiBinary 데이터
        self.action_space = spaces.MultiBinary(1 * 8 * 1024 * 1024)
        self.meta = meta

        # 모델 및 데이터 로더 설정
        self.target_function = target_function  # BinaryNet 모델
        self.trainloader = trainloader          # 학습 데이터 로더

        # 에피소드 설정
        self.max_steps = max_steps
        self.T_PSNR = T_PSNR
        self.T_steps = T_steps

        # 학습 상태
        self.state = None
        self.observation = None
        self.steps = 0
        self.psnr_sustained_steps = 0

        # 학습 데이터셋에서 첫 배치 추출
        self.data_iter = iter(self.trainloader)
        self.target_image = None

    def reset(self, seed=None, options=None):
        try:
            self.target_image = next(self.data_iter)
        except StopIteration:
            self.data_iter = iter(self.trainloader)
            self.target_image = next(self.data_iter)

        self.target_image = self.target_image.cuda()
        with torch.no_grad():
            self.observation = self.target_function(self.target_image).cpu().numpy()

        self.steps = 0
        self.psnr_sustained_steps = 0

        mask = self.create_action_mask(self.observation)
        return self.observation, {"mask": mask}

    def step(self, action, lr=1e-4, z=2e-3):
        with torch.no_grad():
            # 행동을 mask의 shape에 맞게 변환
            action = np.reshape(action, self.observation.shape).astype(np.int8)  # mask와 동일한 shape로 변환

            # 마스크에 따라 고정된 행동 처리
            mask = self.create_action_mask(self.observation)
            action = np.where(mask == 1, 1, action)  # 행동 1로 고정된 곳은 1로 설정
            action = np.where(mask == 0, 0, action)  # 행동 0으로 고정된 곳은 0으로 설정

            # 행동을 텐서로 변환하고 시뮬레이션 실행
            binary = torch.tensor(action).cuda()
            binary.meta = {
                'wl': self.meta['wl'],
                'psize': self.meta['dx'],
            }
            # simulate 호출 시 meta 정보를 직접 전달
            sim = tt.simulate(
                binary,
                z=z,
                psize=self.meta['dx'],  # meta 정보에서 psize 전달
                wl=self.meta['wl']      # meta 정보에서 wl 전달 (필요 시)
            ).abs()**2
            result = torch.mean(sim, dim=1, keepdim=True)


            # 손실 및 보상 계산
            mse = tt.relativeLoss(result, self.target_image, F.mse_loss).detach().cpu().item()
            psnr = tt.relativeLoss(result, self.target_image, tm.get_PSNR).cpu().item()
            reward = -mse

            # 상태 업데이트
            if self.state is None:
                self.state = np.zeros_like(action, dtype=np.float32)
            self.state = np.logical_xor(self.state, action).astype(np.float32)

        # GPU 메모리 해제
        del binary, sim, result
        torch.cuda.empty_cache()

        # 종료 조건
        terminated = psnr >= self.T_PSNR
        truncated = self.steps >= self.max_steps
        self.steps += 1

        # 다음 상태의 마스크 생성
        mask = self.create_action_mask(self.observation)
        info = {"mse": mse, "psnr": psnr, "mask": mask}

        # 관찰값과 상태 결합
        combined_observation = np.stack([self.observation, self.state], axis=0).astype(np.float32)
        return combined_observation, reward, terminated, truncated, info


    def create_action_mask(self, observation):
        """
        관찰값에 따라 행동 마스크 생성.
        관찰값이 0~0.2인 경우 -> 행동 0으로 고정.
        관찰값이 0.8~1인 경우 -> 행동 1로 고정.
        """
        mask = np.ones_like(observation, dtype=np.int8)  # 기본적으로 모든 행동 가능
        mask[observation <= 0.2] = 0  # 관찰값이 0~0.2면 행동 0으로 고정
        mask[observation >= 0.8] = 1  # 관찰값이 0.8~1이면 행동 1로 고정
        return mask


# 데이터 로더 생성
target_dir = '/nfs/dataset/DIV2K/DIV2K_train_HR/DIV2K_train_HR/'
valid_dir = '/nfs/dataset/DIV2K/DIV2K_valid_HR/DIV2K_valid_HR/'
meta = {'wl': 515e-9, 'dx': (7.56e-6, 7.56e-6)}  # 메타 정보
padding = 0

train_dataset = SimpleDataset(target_dir=target_dir, meta=meta, isTrain=True, padding=padding)
valid_dataset = SimpleDataset(target_dir=valid_dir, meta=meta, isTrain=False, padding=padding)

batch_size = 1
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# BinaryNet 모델 로드
model = BinaryNet(num_hologram=8, in_planes=1, convReLU=False, convBN=False,
                  poolReLU=False, poolBN=False, deconvReLU=False, deconvBN=False).cuda()
model.load_state_dict(torch.load('result_v/2024-12-15 14:02:27.770108_pre_reinforce_8_0.002/2024-12-15 14:02:27.770108_pre_reinforce_8_0.002'))
model.eval()

# 마스크 함수 정의
def mask_fn(env):
    return env.create_action_mask(env.observation)

env = BinaryHologramEnv(
    target_function=model,
    trainloader=train_loader,
    meta={'wl': 515e-9, 'dx': (7.56e-6, 7.56e-6)},  # 메타 정보 전달
    max_steps=1000,
    T_PSNR=30,
    T_steps=100
)

env = ActionMasker(env, mask_fn)

# Vectorized 환경 생성
venv = make_vec_env(lambda: env, n_envs=1)
venv = VecNormalize(venv, norm_obs=True, norm_reward=True, clip_obs=10.0)

model = PPO(
    "MlpPolicy",  # 정책 네트워크
    venv,          # 수정된 환경
    verbose=1,
    n_steps=1024,
    batch_size=16,
    gamma=0.99,
    learning_rate=3e-4
)

model.learn(total_timesteps=100000)

# 학습된 모델 저장
model.save(f"ppo_binary_hologram_{current_date}")

torch.Size([1, 8, 1024, 1024])
Using cuda device


/tmp/ipykernel_674012/3270632023.py:349: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('result_v/2024-12-15 14:02:27.770108_pre_reinforce_8_

AttributeError: 'Tensor' object has no attribute 'fft'